In [3]:
# 쥬피터노트북 셀 스타일 조절 
from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 100% !important; }</style>"))

# 경고 메세지 숨기기 
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
# 관련 모듈 임포트
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Yes24 Bestseller
- http://www.yes24.com/24/category/bestseller?CategoryNumber=001&sumgb=06&fetchSize=40&PageNumber=1

## 첫번째 페이지 
- 순위, 책제목, 저자, 정가, 할인가, 책상세URL, 표지URL
- rank   title   writer   price   discount_price   book_url   img_url

In [ ]:
# (1) html 페이지 => html 문자열화
url = 'http://www.yes24.com/24/category/bestseller?CategoryNumber=001&sumgb=06&fetchSize=40&PageNumber=1'
res = requests.get(url)
html_str = res.text
html_str

In [ ]:
# 2) 뷰티풀숩 객체화 
# html 페이지 => 숩객체 
soup = BeautifulSoup(html_str)
soup

In [ ]:
# 3) 소스 파악 
# 크롬개발자 모드 : f12 , ctrl+shift+i
# 순위, 책제목, 저자, 정가, 할인가, 상세URL, 표지URL

In [ ]:
# 테이블 태그 table이가 있는가?
len(soup.select('table'))

In [ ]:
target = soup.select('tr')
len(target)

In [ ]:
# 1위
target[0]

In [ ]:
# 짝수번째 소스 확인
target[3]

In [ ]:
# 4) 홀수번째 tr만 추출
# 순서 선택자 
# 태그명:nth-child(숫자|odd|even)
# 태그는 주로 순서가 있는 스타일 : li, tr, td, th ~
# odd 는 홀수번째, even 은 짝수번째
target = soup.select('tr:nth-child(odd)')
len(target)

In [ ]:
# 2위 컨텐츠인가?
target[0]

In [ ]:
# 개별 정보 추출 - 1위만
base_html = 'http://www.yes24.com/'
rank = target[0].select_one('td').text.replace('.','위')
title = target[0].select_one('img')['alt']
writer = target[0].select_one('div.aupu a').text
temp = target[0].select('p')[1].text
temp = target[0].select('p')[1].text.split()
discount_price = temp[1].replace('→','').split('(')[0]
price = target[0].select_one('div.aupu a')
book_url = base_html + target[0].select_one('a')['href']
img_url = target[0].select_one('img')['src'].replace('S','XL')
price = temp[0]
print('순위 :', rank)
print('제목 :', title)
print('저자 :', writer)
print('정가 :', price)
print('할인가 :', discount_price)
print('상세URL :', book_url)
print('이미지URL :', img_url)

In [ ]:
# 6) 2차원 리스트 구조로 변경
# 빈리스트 생성
book_list = []
base_html = 'http://www.yes24.com/'
for li in target:
    rank = li.select_one('td').text.replace('.','위')
    title = li.select_one('img')['alt']
    writer = li.select_one('div.aupu a').text
    temp = li.select('p')[1].text
    temp = li.select('p')[1].text.split()
    discount_price = temp[1].replace('→','').split('(')[0]
    price = li.select_one('div.aupu a')
    book_url = base_html + li.select_one('a')['href']
    img_url = li.select_one('img')['src'].replace('S','XL')
    price = temp[0]
    print(rank, title, writer, discount_price, price, book_url, img_url)
    # 리스트에 추가
    book_list.append([rank, title, writer, price, discount_price, book_url, img_url])
# 2차원 리스트 결과 확인 
for rank, title, writer, price, discount_price, book_url, img_url in book_list:
    print(rank, title, writer, price, discount_price, book_url, img_url)

In [ ]:
# 데이타프레임 변경
df = pd.DataFrame(book_list, columns = ['rank', 'title', 'writer', 'price', 'discount_price', 'book_url', 'img_url'])
df

## n개의 페이지
- 함수 이용. n은 매개변수로 이용
- 순위, 책제목, 저자, 정가, 할인가, 책상세URL, 표지URL
- rank, title, writer, price, discount_price, book_url, img_url
- yes24(10) 호출하면 1~10페이지의 데이타가 테이타프레임으로 출려되도록 한다.

In [ ]:
# 1) 주소 테스트
# 함수 정의
# n은 페이지 끝번호
def yes24(n):
    for i in range(1, n+1):
        base = 'http://www.yes24.com/24/category/bestseller?CategoryNumber=001&sumgb=06&fetchSize=40&PageNumber='
        url = base + str(i)
        print(url)

In [ ]:
# 함수 호출
yes24(5)

In [ ]:
# 2) 데이타 프레임으로 반환되게 함수 변경
# 함수 정의
# n은 페이지 끝번호
def yes24_2(n):
    #빈리스트 생성
    book_list2 = []
    base_html = 'http://www.yes24.com/'
    for i in range(1, n+1):
        # 접근 페이지 생성
        base = 'http://www.yes24.com/24/category/bestseller?CategoryNumber=001&sumgb=06&fetchSize=40&PageNumber='
        url = base + str(i)
#         print(url)
        # 2-1) 웹 페이지접근 및 페이지 추출
        res = requests.get(url)
        html_str = res.text
        # 2-2) 숲 객체화
        soup = BeautifulSoup(html_str)
        # 2-3) tr 추출
        target = soup.select('tr:nth-child(odd)')
        # 2-4) 2차원 리스트로 저장
        for li in target:
            rank = li.select_one('td').text.replace('.','위')
            title = li.select_one('img')['alt']
            writer = li.select_one('div.aupu a').text
            temp = li.select('p')[1].text
            temp = li.select('p')[1].text.split()
            discount_price = temp[1].replace('→','').split('(')[0]
            price = li.select_one('div.aupu a')
            book_url = base_html + li.select_one('a')['href']
            img_url = li.select_one('img')['src'].replace('S','XL')
            price = temp[0]
#             print(rank, title, writer, discount_price, price, book_url, img_url)
            # 리스트에 추가
            book_list2.append([rank, title, writer, price, discount_price, book_url, img_url])
    df = pd.DataFrame(book_list2, columns = ['rank', 'title', 'writer', 'price', 'discount_price', 'book_url', 'img_url'])
    return df

In [ ]:
# 함수 호출 테스트
yes24_2(5).tail(10)

## 전체 페이지

- 순위, 책제목, 저자, 정가, 할인가, 책상세URL, 표지URL
- rank, title, writer, price, discount_price, book_url, img_url
- 페이지에 책순위 정보가 없는 형태?
- http://www.yes24.com/24/category/bestseller?CategoryNumber=001&sumgb=06&fetchSize=40&PageNumber=26
- 책정보가 없다면 탈출

In [ ]:
# 1) 페이지 출력
# 고정 주소
base = 'http://www.yes24.com/24/category/bestseller?CategoryNumber=001&sumgb=06&fetchSize=40&PageNumber='
page = 1

# 함수 정의
# n은 페이지 끝번호
while True:
    # 주소 생성   
    url = base + str(page)
    # 2-1) 웹 페이지접근 및 페이지 추출
    res = requests.get(url)
    html_str = res.text
    # 2-2) 숲 객체화
    soup = BeautifulSoup(html_str)
    # 2-3) tr 추출
    target = soup.select('tr:nth-child(odd)')
    # 내용이 있다면
    if len(target):
        print(url)
        page += 1
    else:
        print('페이지테스트 종료')
        break

In [ ]:
# 2) 2차원 리스트로 저장하고 테이타 프레임으로 변경
# 고정 주소
base_html = 'http://www.yes24.com/'
base = 'http://www.yes24.com/24/category/bestseller?CategoryNumber=001&sumgb=06&fetchSize=40&PageNumber='
page = 1
book_list3 = []
# 함수 정의
# n은 페이지 끝번호
while True:
    # 1) 주소 생성   
    url = base + str(page)
    # 2) 웹 페이지접근 및 페이지 추출
    res = requests.get(url)
    html_str = res.text
    # 3) 숲 객체화
    soup = BeautifulSoup(html_str)
    # 4) tr 추출
    target = soup.select('tr:nth-child(odd)')
    # 5) 내용이 있다면
    if len(target):
        print(url)
        # 리스트 추가
        for li in target:
            rank = li.select_one('td').text.replace('.','위')
            title = li.select_one('img')['alt']
            writer = li.select_one('div.aupu a').text
            temp = li.select('p')[1].text
            temp = li.select('p')[1].text.split()
            discount_price = temp[1].replace('→','').split('(')[0]
            price = li.select_one('div.aupu a')
            book_url = base_html + li.select_one('a')['href']
            img_url = li.select_one('img')['src'].replace('S','XL')
            price = temp[0]
#             print(rank, title, writer, discount_price, price, book_url, img_url)
            # 리스트에 추가
            book_list3.append([rank, title, writer, price, discount_price, book_url, img_url])

        # 페이지 번호 증가
        page += 1
    else:
        print('페이지테스트 종료')
        break
df = pd.DataFrame(book_list3, columns = ['rank', 'title', 'writer', 'price', 'discount_price', 'book_url', 'img_url'])
df